In [1]:
# Sqlite3 imports
import psycopg2

# Local imports - for ds
# from models import db, Modis
from functions import (
    pull_modis, 
    process_live_data, 
    haversine
)

# DS Logic imports
import time
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt

In [2]:
# Get MODIS Data
dirty_df = pull_modis()
df = process_live_data(dirty_df)

df.head()

pulling modus - sleep for 1
got dataframe  (771, 13)
process_live_data!


,latitude,longitude,brightness,scan,track,satellite,confidence,version,bright_t31,frp,daynight,month,week
0,41.459,-81.674,306.2,1.3,1.1,Terra,67,6.0NRT,280.6,15.8,N,10,43
1,32.152,-101.931,310.4,1.2,1.1,Terra,80,6.0NRT,289.9,14.7,N,10,43
2,32.178,-102.268,304.7,1.2,1.1,Terra,61,6.0NRT,288.2,10.9,N,10,43
3,35.088,-112.073,306.1,4.2,1.9,Terra,67,6.0NRT,282.6,83.0,N,10,43
4,35.091,-112.082,308.5,4.2,1.9,Terra,75,6.0NRT,282.2,95.4,N,10,43


In [14]:
# Credentials
dbname = 'polpmmvo'
user = 'polpmmvo'
password = 'bFk96iZpUbOZwFCDIqK1JaU4e92C5xDx' # Don't commit this!
host = 'salt.db.elephantsql.com'

# Establish connection
pg_conn = psycopg2.connect(dbname=dbname, user=user,
                       password=password, host=host)

# Instantiate cursor
pg_curs = pg_conn.cursor()

In [4]:
drop = """DROP TABLE training"""

In [15]:
# Create table statement
create_training_table = """
CREATE TABLE training (
id SERIAL PRIMARY KEY,
latitude FLOAT,
longitude FLOAT,
brightness FLOAT,
scan FLOAT,
track FLOAT,
satellite VARCHAR(7),
confidence INT,
version VARCHAR(7),
bright_t31 FLOAT,
frp FLOAT,
daynight VARCHAR(7),
month INT,
week INT
);
"""

In [16]:
# Execute table creation
pg_curs.execute(create_training_table)

In [17]:
# Save and finish session
pg_curs.close()
pg_conn.commit()

In [18]:
# Credentials
dbname = 'polpmmvo'
user = 'polpmmvo'
password = 'bFk96iZpUbOZwFCDIqK1JaU4e92C5xDx' # Don't commit this!
host = 'salt.db.elephantsql.com'

# Establish connection
pg_conn = psycopg2.connect(dbname=dbname, user=user,
                       password=password, host=host)

# Instantiate cursor
pg_curs = pg_conn.cursor()

In [19]:
pg_curs.execute("SELECT * FROM training")
pg_curs.fetchall()

# Update

In [21]:
dirty_rows = df.values

In [22]:
# Clean up rows
rows = []

for row in dirty_rows:
    rows.append(tuple(row))

In [23]:
rows[8]

(38.743,
 -122.82799999999999,
 317.6,
 1.1,
 1.1,
 'Terra',
 95,
 '6.0NRT',
 294.0,
 13.7,
 'N',
 10,
 43)

In [24]:
# Loop over the array to write rows in the DB
for row in rows:
    insert = """
    INSERT INTO training
    (latitude, longitude, brightness, scan, track,
     satellite, confidence, version, bright_t31, frp,
     daynight, month, week)
    VALUES 
    """ + str(row) + ';'
    
    pg_curs.execute(insert)

In [25]:
pg_curs.execute("SELECT * FROM training")
pg_curs.fetchall()

[(1,
  41.459,
  -81.674,
  306.2,
  1.3,
  1.1,
  'Terra',
  67,
  '6.0NRT',
  280.6,
  15.8,
  'N',
  10,
  43),
 (2,
  32.152,
  -101.931,
  310.4,
  1.2,
  1.1,
  'Terra',
  80,
  '6.0NRT',
  289.9,
  14.7,
  'N',
  10,
  43),
 (3,
  32.178,
  -102.268,
  304.7,
  1.2,
  1.1,
  'Terra',
  61,
  '6.0NRT',
  288.2,
  10.9,
  'N',
  10,
  43),
 (4,
  35.088,
  -112.073,
  306.1,
  4.2,
  1.9,
  'Terra',
  67,
  '6.0NRT',
  282.6,
  83.0,
  'N',
  10,
  43),
 (5,
  35.091,
  -112.082,
  308.5,
  4.2,
  1.9,
  'Terra',
  75,
  '6.0NRT',
  282.2,
  95.4,
  'N',
  10,
  43),
 (6,
  31.505,
  -102.641,
  302.5,
  1.3,
  1.1,
  'Terra',
  49,
  '6.0NRT',
  288.7,
  9.5,
  'N',
  10,
  43),
 (7,
  38.748,
  -122.789,
  311.4,
  1.1,
  1.1,
  'Terra',
  50,
  '6.0NRT',
  293.1,
  8.1,
  'N',
  10,
  43),
 (8,
  38.744,
  -122.816,
  316.9,
  1.1,
  1.1,
  'Terra',
  94,
  '6.0NRT',
  295.0,
  13.2,
  'N',
  10,
  43),
 (9,
  38.743,
  -122.828,
  317.6,
  1.1,
  1.1,
  'Terra',
  95,
  '6.0NR

In [26]:
# Save and finish session
pg_curs.close()
pg_conn.commit()